In [ ]:
!pip -q install fastapi uvicorn nest_asyncio pyngrok==7.2.0 "pydantic<3" joblib scikit-learn


In [ ]:
NGROK_AUTHTOKEN = "30s47Zrpt0z8JIXJRCLMMklctLS_43GWGBsYeAL5k558po9Xo"  # 있으면 넣고, 없으면 빈 문자열 유지


In [ ]:
import os, joblib, json
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

MODEL_PATH = "/content/model.pkl"

if not os.path.exists(MODEL_PATH):
    # ---- 데모용 간이 이진감성 분류기(positive/negative) 학습 ----
    texts = [
        "너무 행복하고 기분이 좋아!", "정말 만족스러운 경험이었다.", "즐겁고 기대돼.",
        "짜증나고 우울해.", "최악이야 다시는 하기 싫어.", "불안하고 걱정돼."
    ]
    labels = ["positive","positive","positive","negative","negative","negative"]

    pipe = Pipeline([
        ("tfidf", TfidfVectorizer()),
        ("clf", LogisticRegression(max_iter=200))
    ])
    pipe.fit(texts, labels)
    joblib.dump(pipe, MODEL_PATH)

# 로딩(버전 호환 이슈가 있으면 scikit-learn 버전 맞추세요)
pipe = joblib.load(MODEL_PATH)

# 확률 예측 가능 여부 체크
supports_proba = hasattr(pipe, "predict_proba")
label_classes = None
try:
    # 대부분의 sklearn 분류기는 classes_ 보유
    label_classes = list(pipe.classes_)  # 예: ["negative","positive"]
except Exception:
    pass

print("모델 로딩 OK. supports_proba:", supports_proba, "classes:", label_classes)


모델 로딩 OK. supports_proba: True classes: [np.str_('negative'), np.str_('positive')]


In [ ]:
import nest_asyncio
nest_asyncio.apply()

from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
import uvicorn

app = FastAPI(title="TextClassifier API", version="1.0.0", description="PKL 모형 기반 문장 분류 API")
# CORS: 필요시 Custom GPT 도메인 허용(여기선 모두 허용)
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"], allow_credentials=True,
    allow_methods=["*"], allow_headers=["*"],
)

# ------- Request/Response 스키마 -------
class PredictRequest(BaseModel):
    text: str | None = None
    texts: list[str] | None = None  # 배치 요청 지원

class PredictResponse(BaseModel):
    labels: list[str]
    probabilities: list[dict] | None = None  # 각 문장별 {라벨: 확률}

# ------- 헬스체크 -------
@app.get("/health")
def health():
    return {"status": "ok"}

# ------- 예측 엔드포인트 -------
@app.post("/predict", response_model=PredictResponse)
def predict(req: PredictRequest):
    # 단문 또는 배치 둘 다 지원
    inputs = []
    if req.text is not None:
        inputs = [req.text]
    elif req.texts is not None and len(req.texts) > 0:
        inputs = req.texts
    else:
        return PredictResponse(labels=[], probabilities=[])

    preds = pipe.predict(inputs)
    labels = [str(p) for p in preds]

    probas = None
    if supports_proba:
        raw = pipe.predict_proba(inputs)  # shape: (n_samples, n_classes)
        classes = list(pipe.classes_) if label_classes is None else label_classes
        probas = []
        for row in raw:
            probas.append({cls: float(p) for cls, p in zip(classes, row)})

    return PredictResponse(labels=labels, probabilities=probas)

# --- Uvicorn을 백그라운드로 기동하는 헬퍼 ---
def run_uvicorn():
    uvicorn.run(app, host="0.0.0.0", port=8000, log_level="info", access_log=False)


In [ ]:
import threading, time
from pyngrok import ngrok

# ngrok 준비
if NGROK_AUTHTOKEN:
    ngrok.set_auth_token(NGROK_AUTHTOKEN)

# 기존 터널 정리
for t in ngrok.get_tunnels():
    ngrok.disconnect(t.public_url)

# 새 HTTPS 터널
public_tunnel = ngrok.connect(8000, "http")
public_url = public_tunnel.public_url.replace("http://", "https://")  # https로 통일
print("Public URL:", public_url)

# servers 주입용 OpenAPI 오버라이드
from fastapi.openapi.utils import get_openapi

def custom_openapi():
    if app.openapi_schema:
        # 기존에 생성되어 있다면 servers만 바꿔치기
        app.openapi_schema["servers"] = [{"url": public_url}]
        return app.openapi_schema

    openapi_schema = get_openapi(
        title=app.title,
        version=app.version,
        description=app.description,
        routes=app.routes,
    )
    openapi_schema["servers"] = [{"url": public_url}]
    app.openapi_schema = openapi_schema
    return app.openapi_schema

app.openapi = custom_openapi  # 동적 바인딩

# Uvicorn 백그라운드 기동
thread = threading.Thread(target=run_uvicorn, daemon=True)
thread.start()
time.sleep(1.5)  # 서버 부팅 대기

print("Docs:", public_url + "/docs")
print("OpenAPI JSON:", public_url + "/openapi.json")
print("Health:", public_url + "/health")


INFO:     Started server process [456]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Public URL: https://fb975f3114ba.ngrok.app
Docs: https://fb975f3114ba.ngrok.app/docs
OpenAPI JSON: https://fb975f3114ba.ngrok.app/openapi.json
Health: https://fb975f3114ba.ngrok.app/health


In [ ]:
import requests, json

sample = {"texts": ["오늘 정말 행복해!", "불안하고 걱정돼."]}
r = requests.post(public_url + "/predict", headers={"Content-Type":"application/json"}, data=json.dumps(sample))
print(r.status_code)
print(r.json())


200
{'labels': ['positive', 'negative'], 'probabilities': [{'negative': 0.4423641294660283, 'positive': 0.5576358705339717}, {'negative': 0.5989509319175137, 'positive': 0.40104906808248625}]}
